# Stock Strayegy - High Frequency 0002 - Intraday Stock Timing Strategy

# 1. Import Library

In [1]:
import dai
import random
import pandas as pd

# 2. Prepare Strategy Data

In [2]:
sd = "2026-02-01 00:00:00.000"
ed = "2026-02-13 23:59:59.000"

In [3]:
instrument_list = ['600519.SH']

In [4]:
sql = f"""
WITH 
data_origin AS (
    SELECT
        date,
        instrument,
        make_date(year(date), month(date),  day(date)) AS day,
        AVG(close) OVER (PARTITION BY instrument, day ROWS BETWEEN  9 PRECEDING AND CURRENT ROW) AS ma_short,
        AVG(close) OVER (PARTITION BY instrument, day ROWS BETWEEN 29 PRECEDING AND CURRENT ROW) AS ma_long,
        IF(ma_short > ma_long AND m_lag(ma_short,1) < m_lag(ma_long,1), 1, 0) AS signal_buy,
        IF(ma_short < ma_long AND m_lag(ma_short,1) > m_lag(ma_long,1), 1, 0) AS signal_sell,
    FROM cn_stock_bar1m_c
)
SELECT
    date,
    instrument,
    signal_buy,
    signal_sell
FROM data_origin
QUALIFY (signal_buy = 1 OR signal_sell = 1)
ORDER BY date, instrument
"""

In [5]:
df = dai.query(sql, filters={"date":[sd, ed], "instrument":instrument_list}).df()
df

,date,instrument,signal_buy,signal_sell
0,2026-02-02 09:49:00,600519.SH,1,0
1,2026-02-02 10:24:00,600519.SH,1,0
2,2026-02-02 10:30:00,600519.SH,0,1
3,2026-02-02 10:32:00,600519.SH,1,0
4,2026-02-02 10:36:00,600519.SH,0,1
...,...,...,...,...
134,2026-02-13 13:26:00,600519.SH,0,1
135,2026-02-13 14:38:00,600519.SH,1,0
136,2026-02-13 14:42:00,600519.SH,0,1
137,2026-02-13 14:48:00,600519.SH,1,0


In [14]:
def filter_signals(df):

    df_new = df.copy()

    df_new["date"] = pd.to_datetime(df_new["date"])
    df_new["day"] = df_new["date"].dt.date

    df_new["sig"] = None
    df_new.loc[df_new["signal_sell"] == 1, "sig"] = "sell"
    df_new.loc[(df_new["signal_buy"] == 1) & (df_new["sig"].isna()), "sig"] = "buy"

    df_new = df_new[df_new["sig"].notna()].sort_values(["instrument", "day", "date"])

    def pick_group(g: pd.DataFrame) -> pd.DataFrame:
        first_sig = g.iloc[0]["sig"]

        if first_sig == "buy":
            return g[g["sig"] == "buy"].head(1)

        first_sell = g[g["sig"] == "sell"].head(1)

        t0 = first_sell.iloc[0]["date"]
        next_buy = g[(g["sig"] == "buy") & (g["date"] > t0)].head(1)

        return pd.concat([first_sell, next_buy], axis=0)

    out = (
        df_new.groupby(["instrument", "day"], group_keys=False)
           .apply(pick_group)
           .sort_values(["date", "instrument"])
           .reset_index(drop=True)
    )

    return out[["date", "instrument", "signal_buy", "signal_sell"]]

In [15]:
df_filtered = filter_signals(df)
df_filtered

,date,instrument,signal_buy,signal_sell
0,2026-02-02 09:49:00,600519.SH,1,0
1,2026-02-03 09:35:00,600519.SH,1,0
2,2026-02-04 09:31:00,600519.SH,1,0
3,2026-02-05 09:33:00,600519.SH,1,0
4,2026-02-06 09:57:00,600519.SH,0,1
5,2026-02-06 11:11:00,600519.SH,1,0
6,2026-02-09 09:46:00,600519.SH,1,0
7,2026-02-10 09:25:00,600519.SH,1,0
8,2026-02-11 09:33:00,600519.SH,0,1
9,2026-02-11 09:39:00,600519.SH,1,0


## 3. Backtest Strategy

In [41]:
from bigmodule import M

def BigTrader_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    context.holding_days = 1
    context.weight = 1 / len(df['instrument'].unique())

def BigTrader_Before_Trading(context, data):
    context.subscribe_bar(context.instruments, '1m')

def BigTrader_Handle_Tick(context, tick):
    pass

def BigTrader_Handle_Data(context, data):

    dt_now = data.current_dt.strftime("%Y-%m-%d %H:%M:%S")
    df_now = context.data[context.data["date"] == dt_now]

    if dt_now[11:] == "09:25:00":
        for instrument in set(context.get_account_positions().keys()):
            time_diff_days = (data.current_dt - context.get_position(instrument).last_sale_date).days
            if time_diff_days >= context.holding_days:
                context.order_target_percent(instrument, 0)

    if len(df_now) == 0:
        return

    instruments_hold = set(context.get_account_positions().keys())
    instruments_buy  = set(df_now[df_now["signal_buy"]  == 1]["instrument"])
    instruments_sell = set(df_now[df_now["signal_sell"] == 1]["instrument"])

    for instrument in instruments_sell:
        context.order_target_percent(instrument, 0)

    for instrument in instruments_buy:
        context.order_target_percent(instrument, context.weight)
        
def BigTrader_Handle_Trade(context, trade):
    pass

def BigTrader_Handle_Order(context, order):
    pass

def BigTrader_After_Trading(context, data):
    pass

BigTrader = M.bigtrader.v34(
    
    data = df_filtered,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_Initialize,
    before_trading_start = BigTrader_Before_Trading,
    handle_tick          = BigTrader_Handle_Tick,
    handle_data          = BigTrader_Handle_Data,
    handle_trade         = BigTrader_Handle_Trade,
    handle_order         = BigTrader_Handle_Order,
    after_trading        = BigTrader_After_Trading,
    
    capital_base = 1000000  + random.uniform(0, 10),
    frequency="""minute""",
    product_type="""自动""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""open""",
    order_price_field_sell="""open""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader"""
) 

[2026-02-19 12:59:27] [info     ] bigtrader.v34 开始运行 ..
[2026-02-19 12:59:27] [info     ] 2026-02-02, 2026-02-13, , , instruments=1
[2026-02-19 12:59:27] [info     ] bigtrader module V2.2.0
[2026-02-19 12:59:27] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10
[2026-02-19 12:59:28] [info     ] backtest done, raw_perf_ds:dai.DataSource("_69fd7264ad6444e08923468fd391389e")


[2026-02-19 12:59:28] [info     ] bigtrader.v34 运行完成 [0.627s].
